# Import module 

Most codes from https://github.com/carpedm20/DCGAN-tensorflow/blob/master/model.py

In [1]:
import os
import math
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
mnist = input_data.read_data_sets("data/mnist",one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [3]:
def bn(net,scope,is_training):
    return tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope=scope)

def conv(net,wscope,bscope,output_depth = 64, receptive_field=[5,5],stride=[2,2]):
    shape = net.get_shape()
    weights = tf.get_variable(wscope, receptive_field+[shape[-1], output_depth],initializer=tf.truncated_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    net = tf.nn.conv2d(net, weights, strides=[1]+stride+[1], padding='SAME')
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def linear(net,wscope,bscope,output_depth):
    shape = net.get_shape()       
    weights = tf.get_variable(wscope, [shape[-1], output_depth], tf.float32,tf.random_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    out_logit = tf.matmul(net, weights) + biases
    return out_logit

def deconv(net,wscope,bscope,output_shape,receptive_field=[5,5],stride=[2,2]):
    weights = tf.get_variable(wscope, receptive_field+[output_shape[-1], net.get_shape()[-1]],
                                       initializer=tf.random_normal_initializer(stddev=0.02))
    net = tf.nn.conv2d_transpose(net, weights, output_shape=output_shape, strides=[1]+stride+[1])
    biases = tf.get_variable(bscope, [output_shape[-1]], initializer=tf.constant_initializer(0.0))
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def get_shape(net):
    return net.get_shape().as_list()

# Discriminator Network

Here,i will write two "discriminator" function, the "discriminator_y" only has more one "y" parameter than "discriminator". Spliting them into two function in order to easily understanding the structure

In [4]:
def discriminator(inputs, batch_size, is_training=True, reuse=False):
    
    with tf.variable_scope("discriminator", reuse=reuse):
        
        '''1st: Conv -> lrelu'''
        net = conv(inputs,'d_wconv1','d_bconv1',64)
        net = tf.nn.leaky_relu(net)

        '''2nd: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv2','d_bconv2',64*2)
        net = bn(net, scope = 'd_bn2',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        '''3th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv3','d_bconv3',64*4)
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        '''4th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv4','d_bconv4',64*8)
        net = bn(net, scope = 'd_bn4',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        net = tf.reshape(net, [batch_size, -1])   

        '''5th: linear '''
        out_logit = linear(net,"d_wlinear5","d_blinear5",1)
        '''6th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        

        return out, out_logit

In [5]:
def discriminator_y(inputs, y, y_dim, batch_size, is_training=True, reuse=False):
    
    with tf.variable_scope("discriminator_y", reuse=reuse):

        yb = tf.reshape(y,shape=[batch_size,1,1,y_dim])
        '''1st:Concat -> conv -> lrelu'''
        inputs = tf.concat([inputs, yb*tf.ones(get_shape(inputs)[:-1]+[get_shape(yb)[-1]])],axis=3)       
        net = conv(inputs,'d_wconv1','d_bconv1',inputs.get_shape()[-1])
        net = tf.nn.leaky_relu(net)

        '''2nd:Concat -> conv -> bn -> lrelu'''
        net = tf.concat([net, yb*tf.ones(get_shape(net)[:-1]+[get_shape(yb)[-1]] )],axis=3)
        net = conv(inputs,'d_wconv2','d_bconv2',64+y_dim)        
        net = bn(net, scope = 'd_bn2',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        net = tf.reshape(net,[batch_size,-1])
        
        '''3th:Concat -> linear -> bn -> lrelu'''
        net = tf.concat([net,y],axis=1)
        net = linear(net,"d_wlinear3","d_blinear3",1024)        
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.leaky_relu(net)

        '''4th: Concat -> linear'''
        net = tf.concat([net,y],axis=1)
        out_logit = linear(net,"d_wlinear4","d_blinear4",1)  
        '''5th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        

        return out, out_logit

# Generator Network

Here,i will write two "generator" function, the "generator_y" only has more one "y" parameter than "generator". Spliting them into two function in order to easily understanding the structure

In [6]:
def generator( z, batch_size, output_height=28, output_width=28,output_depth=1, is_training=True, reuse=False):
    
    height0,width0 = output_height,output_width
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2)
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2)
    height8,width8 = math.ceil(float(height4)/2),math.ceil(float(width4)/2)
    height16,width16 = math.ceil(float(height8)/2),math.ceil(float(width8)/2)
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''1st: linear -> reshape -> bn -> relu '''
        net = linear(z,"g_wlinear1","g_blinear1",64*8*height16*width16)
        net = tf.reshape(net,[batch_size,height16,width16,64*8])
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        '''2nd: deconv -> bn -> relu '''
        output_shape = [batch_size, height8, width8, 64*4]
        net = deconv(net,'g_wdeconv2','g_bdeconv2',output_shape)
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)

        '''3th: deconv -> bn -> relu '''
        output_shape = [batch_size, height4, width4, 64*2]
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape)
        net = bn(net,scope='g_bn3',is_training=is_training)
        net = tf.nn.relu(net)

        '''4th: deconv -> bn -> relu '''
        output_shape = [batch_size, height2, width2, 64*1]
        net = deconv(net,'g_wdeconv4','g_bdeconv4',output_shape)
        net = bn(net,scope='g_bn4',is_training=is_training)
        net = tf.nn.relu(net)

        '''5th: deconv -> tanh '''
        output_shape = [batch_size, height0, width0, output_depth]
        net = deconv(net,'g_wdeconv5','g_bdeconv5',output_shape)
        out = tf.nn.sigmoid(net)

        return out

In [7]:
def generator_y( z, y, y_dim, batch_size, output_height, output_width,output_depth, is_training=True, reuse=False):
    
    height0,width0 = output_height,output_width
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2)
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2)
    
    with tf.variable_scope("generator_y", reuse=reuse):
        
        yb = tf.reshape(y,[batch_size,1,1,y_dim])
        '''1st:Concat -> linear -> relu '''
        z = tf.concat([z,y],axis=1)
        net = linear(z,"g_wlinear1","g_blinear1",1024)
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        '''2nd: Concat -> linear -> bn -> relu '''
        net = tf.concat([net,y],axis=1)       
        net = linear(net,"g_wlinear2","g_blinear2",1024*2*height4*width4)
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''3th: reshape -> concat -> deconv -> bn -> relu '''
        net = tf.reshape(net,[batch_size,height4,width4,1024*2])
        net = tf.concat([net, yb*tf.ones(get_shape(net)[:-1]+[get_shape(yb)[-1]])],axis=3)
        output_shape = [batch_size, height2, width2, 1024*2]
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape)
        net = bn(net,scope='g_bn3',is_training=is_training)
        net = tf.nn.relu(net)

        '''4th: concat -> deconv -> tanh '''
        net = tf.concat([net, yb*tf.ones(get_shape(net)[:-1]+[get_shape(yb)[-1]])],axis=3)
        output_shape = [batch_size, height0, width0, output_depth]
        net = deconv(net,'g_wdeconv4','g_bdeconv4',output_shape)
        out = tf.nn.tanh(net)

        return out

# set the global parameters.

In [8]:
# some parameters
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 100
y_dim = 10
learning_rate = 0.0002
beta1 = 0.5
y_flag = False
output_height,output_width,output_depth = [28,28,1]
""" Graph Input """
# images
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
#labels
y = tf.placeholder(tf.float32, [batch_size,y_dim], name='y')
# noises
z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z')

# Loss Function 

In [9]:
""" Loss Function """

if not y_flag:
    # output of D for real images
    D_real, D_real_logits = discriminator(inputs, batch_size, is_training=True, reuse=False)
    # output of D for fake images
    G = generator(z, batch_size, is_training=True, reuse=False)
    D_fake, D_fake_logits = discriminator(G, batch_size, is_training=True, reuse=True)
else:  
    D_real, D_real_logits = discriminator_y(inputs,y,y_dim, batch_size, is_training=True, reuse=False)
    G = generator_y(z, y, y_dim, batch_size, output_height, output_width,output_depth, is_training=True, reuse=False)
    D_fake, D_fake_logits = discriminator_y(G,y,y_dim, batch_size, is_training=True, reuse=True)
    
# get loss for discriminator
d_loss_real = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
d_loss_fake = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
d_loss = d_loss_real + d_loss_fake

# get loss for generator
g_loss = tf.reduce_mean(
         tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))


split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [10]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate*5, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
if not y_flag:
    fake_images = generator(z,batch_size, is_training=False, reuse=True)
else:
    fake_images = generator_y(z,y,y_dim,batch_size, output_height, output_width,output_depth, is_training=False, reuse=True)
# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim))
test_labels_onehot = np.zeros([batch_size, y_dim],dtype = np.float32)
test_labels_onehot[np.arange(batch_size), np.arange(batch_size)%int(np.sqrt(batch_size))] = 1.0

# Training a GAN

In [ ]:
start_epoch = 0
start_batch_id = 0


num_steps = 6000
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images,batch_labels = real_image_batch
    batch_images = batch_images.reshape([batch_size,28,28,1]).astype(np.float32)
    batch_labels = real_image_batch[1].astype(np.float32)
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)

    if not y_flag:
        # update D network
        _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, z: batch_z})
        # update G network
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
    else:
        _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, y:batch_labels, z: batch_z})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z,y:batch_labels})
        _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z,y:batch_labels})
        
        
    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(step_ind, 300) == 0:
        if not y_flag:
            samples = sess.run(fake_images, feed_dict={z: sample_z})
        else:
            samples = sess.run(fake_images, feed_dict={z: sample_z,y:test_labels_onehot})
        # put the "batch_size" images into one big canvas
        row = col = int(np.sqrt(batch_size))
        img = np.zeros( [row*28, col*28] )
        for i in range(row):
            for j in range(col):
                img[i*28:(i+1)*28,j*28:(j+1)*28] = samples[i*col+j, :, :, :].squeeze()
        #save the result      
        scipy.misc.imsave('{}.jpg'.format(step_ind),img)

Step: [0] d_loss: 1.32991338, g_loss: 1.33764780
Step: [1] d_loss: 1.59148777, g_loss: 3.77227211
Step: [2] d_loss: 0.34677559, g_loss: 2.70106816
Step: [3] d_loss: 0.38980940, g_loss: 3.71385574
Step: [4] d_loss: 0.25435412, g_loss: 2.51633716
Step: [5] d_loss: 1.20502901, g_loss: 5.37426376
Step: [6] d_loss: 0.90248138, g_loss: 0.17683277
Step: [7] d_loss: 2.89483094, g_loss: 3.77885008
Step: [8] d_loss: 0.78907514, g_loss: 0.57541418
Step: [9] d_loss: 2.11405802, g_loss: 4.19201374
Step: [10] d_loss: 1.69127345, g_loss: 0.11773265
Step: [11] d_loss: 2.93908429, g_loss: 2.80983162
Step: [12] d_loss: 0.95322043, g_loss: 0.52748996
Step: [13] d_loss: 2.24267411, g_loss: 1.73296511
Step: [14] d_loss: 2.10455608, g_loss: 1.40009880
Step: [15] d_loss: 3.30094099, g_loss: 3.36932898
Step: [16] d_loss: 1.42032027, g_loss: 0.26487523
Step: [17] d_loss: 2.68262506, g_loss: 0.42688176
Step: [18] d_loss: 2.73643970, g_loss: 1.08897829
Step: [19] d_loss: 2.16417956, g_loss: 1.90861154
Step: [20]

Step: [163] d_loss: 0.99557734, g_loss: 1.03802252
Step: [164] d_loss: 0.97304714, g_loss: 0.98849827
Step: [165] d_loss: 1.10065794, g_loss: 0.95989025
Step: [166] d_loss: 1.18099451, g_loss: 1.24818587
Step: [167] d_loss: 1.08818817, g_loss: 0.90425825
Step: [168] d_loss: 1.23924732, g_loss: 1.74791121
Step: [169] d_loss: 0.96849906, g_loss: 0.88987440
Step: [170] d_loss: 1.15545988, g_loss: 1.19864988
Step: [171] d_loss: 1.18008268, g_loss: 1.11385059
Step: [172] d_loss: 1.39244163, g_loss: 0.97731543
Step: [173] d_loss: 1.51166916, g_loss: 0.98329759
Step: [174] d_loss: 1.51537025, g_loss: 1.41432703
Step: [175] d_loss: 1.09888065, g_loss: 0.92028904
Step: [176] d_loss: 1.39937341, g_loss: 1.21536875
Step: [177] d_loss: 1.23101699, g_loss: 1.09090912
Step: [178] d_loss: 1.09435165, g_loss: 1.03279853
Step: [179] d_loss: 0.98687041, g_loss: 1.13234925
Step: [180] d_loss: 0.93645352, g_loss: 1.13175166
Step: [181] d_loss: 0.83711672, g_loss: 1.07061982
Step: [182] d_loss: 1.27325416,

Step: [324] d_loss: 1.18382406, g_loss: 0.86003542
Step: [325] d_loss: 1.25065899, g_loss: 0.75421119
Step: [326] d_loss: 1.56037056, g_loss: 0.66353083
Step: [327] d_loss: 1.30410194, g_loss: 0.75888824
Step: [328] d_loss: 1.32621336, g_loss: 0.64462996
Step: [329] d_loss: 1.60927272, g_loss: 0.60262644
Step: [330] d_loss: 1.65050650, g_loss: 0.76254058
Step: [331] d_loss: 1.38738084, g_loss: 0.87140197
Step: [332] d_loss: 1.35635638, g_loss: 0.80082595
Step: [333] d_loss: 1.30762959, g_loss: 0.74549979
Step: [334] d_loss: 1.26398706, g_loss: 0.78132021
Step: [335] d_loss: 1.41272974, g_loss: 0.76076770
Step: [336] d_loss: 1.29337597, g_loss: 0.76921171
Step: [337] d_loss: 1.36765146, g_loss: 0.73080438
Step: [338] d_loss: 1.35786033, g_loss: 0.73576140
Step: [339] d_loss: 1.33533216, g_loss: 0.72925848
Step: [340] d_loss: 1.32906353, g_loss: 0.68336517
Step: [341] d_loss: 1.33934855, g_loss: 0.73104161
Step: [342] d_loss: 1.33172846, g_loss: 0.73150134
Step: [343] d_loss: 1.45468831,